In [19]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd

In [9]:
class spectra:
    def __init__(self, filename):
        self.chan, self.energy, self.count= np.loadtxt('spectra/Background_16_10_23_Mon_14_50_edited.csv',delimiter = ',',skiprows=3,unpack = True)

chan_bg_spec,E_bg_spec,C_bg_spec = np.loadtxt('spectra/Background_16_10_23_Mon_14_50.csv',delimiter = ',',skiprows=7,unpack = True)
peaks, _ =signal.find_peaks(C_bg_spec)

In [20]:
bg = pd.read_csv('spectra/Background_16_10_23_Mon_14_50.csv',delimiter = ',')

In [ ]:
pk_bg = np.loadtxt('spectra/Background_16_10_23_Mon_14_50.csv',delimiter = ',',skiprows=7,unpack = True)

In [21]:
bg

,,Start Time,Mon Oct 16 2023 10:14:18 GMT+0100 (BST)
Energy calibration,Offset: -0.3439140022,Slope: 0.6775230169,Quadratic: 8.387880257e-8
Live Time (s),353069.86,NaN,NaN
Real Time (s),354394.61,NaN,NaN
Elapsed Computational,0,NaN,NaN
Spectrum,NaN,NaN,NaN
...,...,...,...
4092,2773.4851199746568,1,NaN
4093,2774.1633296234345,3,NaN
4094,2774.84153943997,6,NaN
4095,2775.5197494242634,5,NaN
